# [Sentiment Analysis Shared Task](https://github.com/blp-workshop/blp_task2) at [BLP Workshop](https://blp-workshop.github.io/) @EMNLP 2023

The main objective of this task is to detect the sentiment associated within a given text. This is a multi-class classification task that involves determining whether the sentiment expressed in the text is Positive, Negative, Neutral.



### Downloading dataset from github

In [1]:
!wget https://raw.githubusercontent.com/blp-workshop/blp_task2/main/data/blp23_sentiment_train.tsv
!wget https://raw.githubusercontent.com/blp-workshop/blp_task2/main/data/blp23_sentiment_dev.tsv
!wget https://raw.githubusercontent.com/blp-workshop/blp_task2/main/data/blp23_sentiment_dev_test.tsv

--2023-08-14 19:19:25--  https://raw.githubusercontent.com/blp-workshop/blp_task2/main/data/blp23_sentiment_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8607028 (8.2M) [text/plain]
Saving to: ‘blp23_sentiment_train.tsv’

blp23_sentiment_tra 100%[===================>]   8.21M  --.-KB/s    in 0.06s   

2023-08-14 19:19:25 (140 MB/s) - ‘blp23_sentiment_train.tsv’ saved [8607028/8607028]

--2023-08-14 19:19:26--  https://raw.githubusercontent.com/blp-workshop/blp_task2/main/data/blp23_sentiment_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaitin

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate
!pip install git+https://github.com/csebuetnlp/normalizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-9hj8evyv
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-9hj8evyv
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl 

#### importing required libraries and setting up logger

In [3]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers import AutoModelForPreTraining, AutoTokenizer
from normalizer import normalize
from transformers import AutoModelForSequenceClassification


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

### Defining the training, validation, and test data

In [4]:
train_file = 'blp23_sentiment_train.tsv'
validation_file = 'blp23_sentiment_dev.tsv'
test_file = 'blp23_sentiment_dev_test.tsv'

### Setting up the training parameters

In [5]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no"
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

In [6]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

#### Defining the Model

In [7]:
model_name = "csebuetnlp/banglishbert"

#### setting the random seed

In [8]:
set_seed(training_args.seed)

#### Loading data files

In [9]:
l2id = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
train_df = pd.read_csv(train_file, sep='\t')
train_df['label'] = train_df['label'].map(l2id)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

##### Extracting number of unique labels

In [10]:
# Labels
label_list = raw_datasets["train"].unique("label")
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

### Loading Pretrained Configuration, Tokenizer and Model

In [11]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

[INFO|configuration_utils.py:669] 2023-08-14 19:20:55,053 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/config.json
[INFO|configuration_utils.py:725] 2023-08-14 19:20:55,064 >> Model config ElectraConfig {
  "_name_or_path": "csebuetnlp/banglishbert",
  "_num_labels": 2,
  "amp": true,
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_

[INFO|configuration_utils.py:669] 2023-08-14 19:20:55,265 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/config.json
[INFO|configuration_utils.py:725] 2023-08-14 19:20:55,267 >> Model config ElectraConfig {
  "_name_or_path": "csebuetnlp/banglishbert",
  "_num_labels": 2,
  "amp": true,
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type"

[INFO|tokenization_utils_base.py:1823] 2023-08-14 19:20:55,987 >> loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/vocab.txt
[INFO|tokenization_utils_base.py:1823] 2023-08-14 19:20:55,988 >> loading file tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:1823] 2023-08-14 19:20:55,989 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1823] 2023-08-14 19:20:55,990 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/special_tokens_map.json
[INFO|tokenization_utils_base.py:1823] 2023-08-14 19:20:55,991 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/tokenizer_config.json
[INFO|configuration_utils.py:669] 2023-08

[INFO|modeling_utils.py:2578] 2023-08-14 19:21:00,233 >> loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglishbert/snapshots/88f2777acd65160b2a6c07e5ffef2d232daadf87/pytorch_model.bin
[WARNING|modeling_utils.py:3285] 2023-08-14 19:21:01,608 >> Some weights of the model checkpoint at csebuetnlp/banglishbert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to b

#### Preprocessing the raw_datasets

In [12]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/36 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

#### Finalize the training data for training the model

In [13]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [14]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 35266
})

#### Finalize the development/evaluation data for evaluating the model

In [15]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [16]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [17]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

#### Get the metric function `accuracy`

In [18]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [19]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [20]:
data_collator = default_data_collator

#### Initialize our Trainer

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

#### Training our model

In [22]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[INFO|trainer.py:1786] 2023-08-14 19:21:16,787 >> ***** Running training *****
[INFO|trainer.py:1787] 2023-08-14 19:21:16,790 >>   Num examples = 35,266
[INFO|trainer.py:1788] 2023-08-14 19:21:16,793 >>   Num Epochs = 30
[INFO|trainer.py:1789] 2023-08-14 19:21:16,795 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:1790] 2023-08-14 19:21:16,798 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:1791] 2023-08-14 19:21:16,799 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1792] 2023-08-14 19:21:16,800 >>   Total optimization steps = 66,150
[INFO|trainer.py:1793] 2023-08-14 19:21:16,806 >>   Number of trainable parameters = 

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.852600
1000,0.789000
1500,0.755800
2000,0.746400
2500,0.660600
3000,0.591300
3500,0.608300
4000,0.595900
4500,0.561500
5000,0.407600


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Saving the tokenizer too for easy upload

In [28]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:2926] 2023-08-14 23:18:22,145 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:458] 2023-08-14 23:18:22,152 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:1853] 2023-08-14 23:18:23,136 >> Model weights saved in ./distilBERT_m/pytorch_model.bin
[INFO|tokenization_utils_base.py:2194] 2023-08-14 23:18:23,147 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2201] 2023-08-14 23:18:23,152 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                   =       30.0
  eval_accuracy           =     0.7021
  eval_loss               =     3.2142
  eval_runtime            = 0:00:16.33
  eval_samples            =       3934
  eval_samples_per_second =    240.822
  eval_steps_per_second   =     15.059


#### Evaluating our model on validation/development data

In [29]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

[INFO|trainer.py:3200] 2023-08-14 23:18:29,243 >> ***** Running Evaluation *****
[INFO|trainer.py:3202] 2023-08-14 23:18:29,246 >>   Num examples = 3934
[INFO|trainer.py:3205] 2023-08-14 23:18:29,247 >>   Batch size = 16


***** eval metrics *****
  epoch                   =       30.0
  eval_accuracy           =     0.7021
  eval_loss               =     3.2142
  eval_runtime            = 0:00:16.17
  eval_samples            =       3934
  eval_samples_per_second =    243.277
  eval_steps_per_second   =     15.213


### Predecting the test data

In [30]:
id2l = {0:'Negative', 1:'Neutral', 2:'Positive'}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"predict_results.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\n")

KeyError: "Column id not in the dataset. Current columns in the dataset: ['text', 'input_ids', 'token_type_ids', 'attention_mask']"

In [31]:
ids[0]

30670

#### Saving the model into card

In [32]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:451] 2023-08-14 23:18:55,932 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7020844221115112}]}
